In [1]:
import pandas as pd
import math
import numpy as np
import os

import time
from random import sample
from statistics import mean

# import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import scipy

from scipy.stats import skew

In [2]:
#FUNCTION DEFINITIONS
def mean_confidence_interval(data, confidence=0.95):
    '''
    Function to get confidence interval of list of data
    '''
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m-h, m+h

#Decided against using datetime, so just wrote some custom functions for the int type yyyymm
def monthminus(i):
    '''
    get the previous month
    '''
    if(i == 192606):
        return i
    elif(i%100 == 1):
        year = i//100
        return (year-1)*100 + 12
    else:
        return i-1   


def monthdifference(dob, curr):
    '''
    get difference in months between two dates
    '''
    ydob = dob//100
    ycur = curr//100
    mdob = dob%100
    mcur = curr%100
    return (ycur-ydob)*12 + mcur-mdob


starty = 1926  
startm = 7
endy = 2016
N = 100
startd = starty*100 + startm  
    

In [3]:
#DATA LOADING
data = pd.read_csv('./newdata2.csv') #data only containing relevant share codes
fulldata = pd.read_csv('./fulldata2.csv') #data containing all share codes (to accomodate reclassification)

# Data Cleaning and Preprocessing

Implementing a dataframe matrix, with indices as PERMNO's, and columns as dates to quickly access key stock variables without having to parse through entire database

In [4]:
#DATA PREPROCESSING - limited sharecodes

fulldata = fulldata[~fulldata.SHRCD.isna()]

codereplace = [500,520,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,580,584]
retcodereplace = ['A','S','T','P']
data.loc[data['DLSTCD'].isin(codereplace) & data['DLRET'].isin(retcodereplace), 'DLRET' ] = -0.3
data.loc[data['DLRET'].isin(retcodereplace), 'DLRET'] = -.9999



data['RET'] = data['RET'].fillna(0.00)
data['RETX'] = data['RETX'].fillna(0.00)
data['DLRET'] = data['DLRET'].fillna(0.00)
data.loc[data['RET'].isin(['A','B','C','D','E']), 'RET'] = 0.00
data.loc[data['RETX'].isin(['A','B','C','D','E']), 'RETX'] = 0.00
# data = data.loc[~data['RET'].isin(['B','C'])]
data = data.drop_duplicates(subset=['PERMNO','date','RET'], keep = 'first')


data['date'] = data['date']//100  #only care about monthly returns
# data['date'] = pd.to_datetime(data['date'], format="%Y%m")
# data['date']
    
#ADDING IN DOB
dateofbirths = data.sort_values(by = 'date').drop_duplicates(subset = ['PERMNO'],keep='first')
dateofbirths = dateofbirths.rename(columns={'date':'dateofbirth'})
data = data.merge(dateofbirths[['PERMNO','dateofbirth']], on = 'PERMNO', how = 'left')

#adding in Date of Delisting    
delistings = data.loc[~data['DLSTCD'].isna()]
delistings=  delistings.rename(columns = {'date':'dateofdelisting'})
data = data.merge(delistings[['PERMNO','dateofdelisting']], on = 'PERMNO', how = 'left')


#delete rows  that are useless delisting returns
indexNames = list(data[data['PRC'].isna() & data['SHROUT'].isna() & (data['dateofdelisting']<data['date'])].index)
data.drop(indexNames, inplace = True)


#get Age by month
data['Age'] = (monthdifference(data['dateofbirth'],data['date'])+1)
#get Size
data['Size'] = data['SHROUT'] * abs(data['PRC'])

data['Size'].fillna(method='ffill', inplace=True)

data['RET'] = data['RET'].apply(pd.to_numeric)
data['RETX'] = data['RETX'].apply(pd.to_numeric)

data['DLRET'] = data['DLRET'].apply(pd.to_numeric)

ret_max1 = data[['date','RET','PERMNO']]

ret_max = ret_max1.pivot(index = 'PERMNO', columns = 'date', values = 'RET')  #matrix of returns



In [5]:
#DATA PREPROCESSING - all sharecodes


listofPERMNO = data.PERMNO.unique()

#filter full data so that only PERMNOs that were at one time 10,11, or 12
fulldata = fulldata.loc[fulldata['PERMNO'].isin(listofPERMNO)]
fulldata['date'] = fulldata['date']//100


fulldata.loc[fulldata['DLSTCD'].isin(codereplace) & fulldata['DLRET'].isin(retcodereplace), 'DLRET' ] = -0.3
fulldata.loc[fulldata['DLRET'].isin(retcodereplace), 'DLRET'] = -.9999
# fulldata.loc[~fulldaDLSTCDta['DLRET'].isna(), 'DLRET'] = -.9999
fulldata['RET'] = fulldata['RET'].fillna(0.00)
fulldata['RETX'] = fulldata['RETX'].fillna(0.00)
fulldata['DLRET'] = fulldata['DLRET'].fillna(0.00)
fulldata.loc[fulldata['RET'].isin(['A','B','C','D','E']), 'RET'] = 0.00
fulldata.loc[fulldata['RETX'].isin(['A','B','C','D','E']), 'RETX'] = 0.00
# fulldata = fulldata.loc[~data['RET'].isin(['B','C'])]
fulldata = fulldata.drop_duplicates(subset=['PERMNO','date','RET'], keep = 'first')


#ADDING IN DOB
dateofbirths = fulldata.sort_values(by = 'date').drop_duplicates(subset = ['PERMNO'],keep='first')
dateofbirths = dateofbirths.rename(columns={'date':'dateofbirth'})
fulldata = fulldata.merge(dateofbirths[['PERMNO','dateofbirth']], on = 'PERMNO', how = 'left')

#adding in Date of Delisting    
delistings = fulldata.loc[~fulldata['DLSTCD'].isna()]
delistings=  delistings.rename(columns = {'date':'dateofdelisting'})
fulldata = fulldata.merge(delistings[['PERMNO','dateofdelisting']], on = 'PERMNO', how = 'left')

#delete useless delisting returns
# print(fulldata['dateofdelisting'])
indexNames = list(fulldata[fulldata['PRC'].isna() & (fulldata['dateofdelisting']<fulldata['date'])].index)
fulldata.drop(indexNames, inplace = True)

#indexNames = list(fulldata[fulldata['PRC'].isna() & fulldata['dateofdelisting'].isna()].index)
indexNames = list(fulldata[fulldata['date']< 192606].index)

fulldata.drop(indexNames, inplace = True)


fulldata.loc[fulldata.dateofbirth < 192606, 'dateofbirth'] = 192606

fulldata['Age'] = (monthdifference(fulldata['dateofbirth'],fulldata['date'])+1)

fulldata['RET'] = fulldata['RET'].apply(pd.to_numeric)
fulldata['RETX'] = fulldata['RETX'].apply(pd.to_numeric)
fulldata['DLRET'] = fulldata['DLRET'].apply(pd.to_numeric)

ret_max1 = fulldata[['date','RET','PERMNO']]
ret_max2 = fulldata[['date','RETX','PERMNO']]
dlret_max1 = fulldata[['date','DLRET','PERMNO']]
age_max1 = fulldata[['date','Age','PERMNO']]
price_max1 = fulldata[['date', 'PRC','PERMNO']]
# div_max1 = fulldata[['date','DIVAMT','PERMNO']]
size_max1 = fulldata[['date','Size','PERMNO']]

full_ret_max = ret_max1.pivot(index = 'PERMNO', columns = 'date', values = 'RET')
nodiv_full_ret_max = ret_max2.pivot(index = 'PERMNO', columns = 'date', values = 'RETX')
dlret_max = dlret_max1.pivot(index = 'PERMNO', columns = 'date', values = 'DLRET')    
age_max =  age_max1.pivot(index = 'PERMNO', columns = 'date', values = 'Age')
price_max = price_max1.pivot(index = 'PERMNO', columns = 'date', values = 'PRC')
# div_max = div_max1.pivot(index='PERMNO',columns='date',values='DIVAMT')
size_max = size_max1.pivot(index = 'PERMNO', columns = 'date', values = 'Size')

KeyError: "['Size'] not in index"

# Creating ListofAvailable Dict

Have a dict of lists of available stocks in each month

In [ ]:
month = startm
year=  starty

listofdates = []
listofavailable = {}
listofyears = []
fulllistofavailable = {}
while(year<=endy):
    listofyears.append(year)
    while(month <= 12):
        curdate = year*100 + month
        listofdates.append(curdate)
        y = ret_max.index[(ret_max[curdate].notnull())].tolist()
        x = full_ret_max.index[(full_ret_max[curdate].notnull())].tolist()
        listofavailable[curdate] = y
        fulllistofavailable[curdate] = x
        month += 1
    month = 1
    year += 1

# Rebalanced Portfolio Method Simulation

This portfolio selected N stocks in the starting month such that each stock is of equal weight in the portfolio. After each month the stocks are rebalanced so that the equal weighting is maintained. Delisted stocks are replaced.

In [ ]:
simulation_num = 100



#Rebalanced Simulation
ew_ret = {} #dictionary of monthly returns   
exits = []
entrants = []
ret_list = []
# in_list = []
agedict = {}
ew_ret_list = {}
year = starty
month = startm

while(year <= endy):
    while(month<= 12):
        curdate=  year*100 + month
        agedict[curdate] = []
        month+=1
    month = 1
    year += 1
    
    
n_list = [5,25,50,100]
#dictionaries to keep track of variables on a month by month basis
ret_dict = {'rebalanced':{},'bootstrapped':{}}
turnover_dict = {'rebalanced':{},'bootstrapped':{}}
turnover_dict_list = {'rebalanced':{},'bootstrapped':{}}
ret_dict_list = {'rebalanced':{},'bootstrapped':{}}
age_dict_list = {'rebalanced':{},'bootstrapped':{}}
for strat in ret_dict:
    for w in n_list:
        ret_dict[strat][w] = []
        turnover_dict[strat][w] = []
        ret_dict_list[strat][w] = {}
        age_dict_list[strat][w] = {}
        size_dict_list[strat][w] = {}
        turnover_dict_list[strat][w] = {}
        for i in listofdates:
            turnover_dict_list[strat][w][i] = []
            ret_dict_list[strat][w][i] = []
            age_dict_list[strat][w][i] = []
            size_dict_list[strat][w][i] = []
            age_dict_list[strat][w][i] = []

strat = 'rebalanced'
for N in n_list:
    for j in range(0,simulation_num):
        ew_ret = {}
        turnover = [] #list to keep track of turnover month by month
#         exits = []
#         entrants = []
#         in_list = []
        total_ret = 1 #initalize return as zero
        start = time.time()
        for i in listofdates:
            if(i == startd):
                prev_sample = sample(listofavailable[i],N) #Take first sample
                ew_ret[i] = sum(ret_max.loc[prev_sample,i])
                ew_ret[i] += sum(dlret_max.loc[prev_sample,i])
                ew_ret[i] = 1 + ew_ret[i] /N #calculate monthly return
                ret_dict_list[strat][N][i].append(ew_ret[i])
                age_dict_list[strat][N][i].append(mean(age_max.loc[prev_sample,i]))
                size_dict_list[strat][N][i].append(mean(size_max.loc[prev_sample,i]))
                turn = np.mean(np.abs(ret_max.loc[prev_sample,i].values))
                turnover_dict_list[strat][N][i].append(turn)
#                 turnover.append(turn)
            else:
                    
                cur_sample = list(set(prev_sample)& set(listofavailable[i])) #find items in next date

                diff = list(set(prev_sample) - set(cur_sample)) #find items that have exited either due to reclassification or delisting
                reclassified = list(set(fulllistofavailable[i]) & set(diff)) #find items that have been reclassified

                cur_sample = cur_sample + reclassified #add back in items that have been reclassified
                
                percent_prc_chg = np.abs(nodiv_full_ret_max.loc[cur_sample,i])

                exitnum = N - len(cur_sample) #how many stocks have exited
                turn = (np.mean(percent_prc_chg) * len(cur_sample) + exitnum)/N
                turnover_dict_list[strat][N][i].append(turn)
#                 turnover.append(turn)

                filling = sample(set(listofavailable[i])-set(cur_sample), exitnum) #sample in to fill exited stocks

                if(len(filling)>0):  #add new samples to portfolio
                    cur_sample.extend(filling) 
                    
                normal_returns = full_ret_max.loc[cur_sample,i]
                delisting_returns = dlret_max.loc[cur_sample,i]


                ew_ret[i] = sum(normal_returns) #get return matrix
                ew_ret[i] += sum(delisting_returns) #get delisting returns
                ew_ret[i] = 1 + ew_ret[i]/N #return for a given month
                
                ret_dict_list[strat][N][i].append(ew_ret[i]) #append to monthly return list
                
                averageage = mean(age_max.loc[prev_sample,i])
                age_dict_list[strat][N][i].append(mean(age_max.loc[prev_sample,i]))
                size_dict_list[strat][N][i].append(mean(size_max.loc[prev_sample,i]))
                
                prev_sample = cur_sample
            total_ret = total_ret * ew_ret[i] #keep a running track of total return
        end = time.time()
    
        annualizedret = total_ret ** (1/((len(listofdates)-1)/12)) -1
        print(f'It took {end-start} seconds for simulation #{j} of portfolio size {N}, Ann. Ret. = {annualizedret}')

        ret_dict[strat][N].append(annualizedret)
        total_turnover = sum(turnover)/len(turnover)
        turnover_dict[strat][N].append(total_turnover)


# Bootstrapped Portfolio Simulation

This portfolio selects N stocks with equal weight in the first month. It then selects N new stocks every new month. Turnover is obviously very high with this method.

In [ ]:
#Bootstrapped Simulation

simulation_num = 100

boot_ret_list = []

strat = 'bootstrapped'

for j in range(0,simulation_num):
    print(j)
    total_ret = 1.00
    for i in listofdates:
        prev_sample = sample(listofavailable[i],N)
        ew_ret[i] = sum(ret_max.loc[prev_sample,i])
        ew_ret[i] += sum(dlret_max.loc[prev_sample,i])
        ew_ret[i] = 1 + ew_ret[i] /N
        averageage = mean(age_max.loc[prev_sample,i])
        age_dict_list[strat][N][i].append(averageage)
        ret_dict_list[strat][N][i].append(ew_ret[i])
        size_dict_list[strat][N][i].append(ew_ret[i])
        turnover_dict_list[strat][N][i].append(1)
        total_ret = total_ret * ew_ret[i]
        
    annualizedret = total_ret ** (1/(len(listofdates)/12)) -1
    ret_list[strat].append(annualizedret)
            



# Return Profile of Bootstrapped vs Rebalanced Simulation

It is clear that the rebalanced portfolio outperforms the bootstrapped portfolio 

In [ ]:
labels = []
rect_list = []
i = 0
width = 0.35  # the width of the bars

for strat in ret_dict_list:
    for size in n_list:
        if i == 0:
            labels.append(size)
        print(strat)
    #     rounded_avg_ret = [ round(elem, 2) for elem in ret_dict[size]]
        mean_ret = mean(ret_dict[strat][size])
        print(f'Portfolio Size: {size} - Mean Return:{mean_ret}, Median Return: {median(ret_dict[strat][size])} Skew Return:{skew(ret_dict[strat][size])},Mean Turnover: {mean(turnover_dict[strat][size])},Skew Turnover: {skew(turnover_dict[strat][size])}')
        rect_list.append(ax.bar(x-width/len(n_list),mean_ret,width,label=strat)
    i += 1
    


x = np.arange(len(labels))  # the label locations

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='Men')
rects2 = ax.bar(x + width/2, women_means, width, label='Women')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Mean Annualized Return')
ax.set_title('Return Comparison by Portfolio Size')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

for r in rect_list:
    autolabel(r)

fig.tight_layout()

plt.show()

# Age, Size, and Monthly Return Through Time


In [ ]:
for N in n_list:
    

# Turnover Through Time

We can see that in periods of volatility, such as 2008 and the late 1990's turnover spikes. As stocks move around more more wealth from the rebalanced portfolios must be sacrificed.

In [ ]:
    
import json
from datetime import datetime
import matplotlib.dates as mdates
from statistics import median    
    

avg_dict_ret = {}
avg_dict_turn = {}
for i in listofdates:
    avg_dict_ret[i] = sum(ret_dict_list[100][i])/len(ret_dict_list[100][i])
    avg_dict_turn[i] = sum(turnover_dict_list[100][i])/len(turnover_dict_list[100][i])

    

dates = list(avg_dict_turn.keys())           # list() needed for python 3.x
for l in range(0,len(dates)):
    date = datetime.strptime(str(dates[l]), '%Y%m').date()
    dates[l] = date
AVG_turnover = list(avg_dict_turn.values())        # ditto
rounded_avg_turnover = [ round(elem, 2) for elem in AVG_turnover]
fig, ax = plt.subplots()

#set locator to every decade
locator = mdates.YearLocator(10)

ax.xaxis.set_major_locator(locator)



ax.format_xdata = mdates.DateFormatter('%Y-%m')
ax.grid(True)

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()


ax.plot_date(dates, AVG_turnover, '-')


fig.set_figheight(15)
fig.set_figwidth(15)

# fig.suptitle('Turnover Through Time', fontsize=20)
ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)

fig.suptitle('Rebalanced Portfolio Overtime')

# fig.savefig('turnover.png')

# Turnover vs annualized return for rebalanced portfolios

There does not appear to be a relationship between increased turnover and increased annualized return.

In [ ]:
from sklearn.linear_model import LinearRegression

x = np.asarray(ret_dict[100])
y = np.asarray(turnover_dict[100])
fig, ax = plt.subplots()
ax.scatter(x,y)

ax.set_xlabel('Annualized Return', fontsize = 20)
ax.set_ylabel('Turnover', fontsize = 20)
fig.set_figheight(10)
fig.set_figwidth(10)

ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)

# fig.savefig('scatter.png')

